<a href="https://www.kaggle.com/hrialan/transfer-learning?scriptVersionId=89824664" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
running_on_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') != 'Localhost'

if running_on_kaggle:
    print('Kaggle environment detected')
    # !nvidia-smi
    data_dir = '/kaggle/input/food41/images/'
else:
    print('Running on localhost')
    data_dir = './dataset/food/images/'
    !pip install --quiet -r requirements.txt
    !python download_data.py

Kaggle environment detected


In [3]:
batch_size = 32

img_height = 224
img_width = 224

In [4]:
train_data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.2)

test_data_generator = ImageDataGenerator(
    rescale = 1./255)

In [5]:
train_data = train_data_generator.flow_from_directory(
    data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training')

val_data = test_data_generator.flow_from_directory(
    data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode='categorical',
    subset = 'validation')

Found 80800 images belonging to 101 classes.
Found 0 images belonging to 101 classes.


# ResNet50V2

In [6]:
"""
ResNet50V2 = tf.keras.applications.resnet_v2.ResNet50V2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes=101
)

for layer in ResNet50V2.layers:
    layer.trainable = False
"""

"\nResNet50V2 = tf.keras.applications.resnet_v2.ResNet50V2(\n    input_shape=(224, 224, 3),\n    include_top=False,\n    weights='imagenet',\n    pooling='avg',\n    classes=101\n)\n\nfor layer in ResNet50V2.layers:\n    layer.trainable = False\n"

In [7]:
"""
model_ResNet = tf.keras.Sequential([
    ResNet50V2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax')
    ])

model_ResNet.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(lr=.0001), 
    metrics=['accuracy'])
"""

"\nmodel_ResNet = tf.keras.Sequential([\n    ResNet50V2,\n    tf.keras.layers.Flatten(),\n    tf.keras.layers.Dense(101, activation = 'softmax')\n    ])\n\nmodel_ResNet.compile(\n    loss='categorical_crossentropy', \n    optimizer=tf.keras.optimizers.Adam(lr=.0001), \n    metrics=['accuracy'])\n"

In [8]:
"""
epochs=10
history = model_ResNet.fit_generator(
  train_data,
  validation_data=val_data,
  epochs=epochs
)
"""

'\nepochs=10\nhistory = model_ResNet.fit_generator(\n  train_data,\n  validation_data=val_data,\n  epochs=epochs\n)\n'

# InceptionResnetV2

In [9]:
"""
InceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes=101
)

for layer in InceptionResNetV2.layers:
    layer.trainable = False
    
x = InceptionResNetV2.layers[-1].output
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)

output = layers.Dense(101, activation='softmax')(x)
model = models.Model(InceptionResNetV2.input, output)

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=.0001), 
              metrics=['accuracy'])
"""

"\nInceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(\n    input_shape=(224, 224, 3),\n    include_top=False,\n    weights='imagenet',\n    pooling='avg',\n    classes=101\n)\n\nfor layer in InceptionResNetV2.layers:\n    layer.trainable = False\n    \nx = InceptionResNetV2.layers[-1].output\nx = layers.BatchNormalization()(x)\nx = layers.Dropout(0.2)(x)\nx = layers.Dense(128, activation='relu')(x)\nx = layers.Dropout(0.2)(x)\n\noutput = layers.Dense(101, activation='softmax')(x)\nmodel = models.Model(InceptionResNetV2.input, output)\n\nmodel.compile(loss='categorical_crossentropy', \n              optimizer=tf.keras.optimizers.Adam(lr=.0001), \n              metrics=['accuracy'])\n"

In [10]:
# model.summary()

In [11]:
"""
epochs=10
history = model.fit_generator(
  train_data,
  validation_data=val_data,
  epochs=epochs
)
"""

'\nepochs=10\nhistory = model.fit_generator(\n  train_data,\n  validation_data=val_data,\n  epochs=epochs\n)\n'

# InceptionV3

In [12]:
InceptionV3 = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    classes=101)

# for layer in InceptionV3.layers:
#     layer.trainable = False
    
x = InceptionV3.output
x = layers.AveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
output = layers.Dense(101, activation='softmax')(x)
model = models.Model(InceptionV3.input, output)

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=.9),
              metrics=['accuracy'])

2022-03-11 09:19:20.335409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 09:19:20.441716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 09:19:20.442478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 09:19:20.443682: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 1s 0us/step


In [13]:
epochs=10
history = model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs
)

2022-03-11 09:19:26.950646: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-03-11 09:19:33.936934: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2525/2525 [==============================] - 1664s 654ms/step - loss: 2.5727 - accuracy: 0.3856
Epoch 2/10
2525/2525 [==============================] - 1305s 517ms/step - loss: 1.5592 - accuracy: 0.5978
Epoch 3/10
2525/2525 [==============================] - 1328s 526ms/step - loss: 1.2964 - accuracy: 0.6602
Epoch 4/10
2525/2525 [==============================] - 1348s 534ms/step - loss: 1.1443 - accuracy: 0.6978
Epoch 5/10
2525/2525 [==============================] - 1367s 541ms/step - loss: 1.0407 - accuracy: 0.7225
Epoch 6/10
2525/2525 [==============================] - 1350s 535ms/step - loss: 0.9514 - accuracy: 0.7433
Epoch 7/10
2525/2525 [==============================] - 1332s 527ms/step - loss: 0.8831 - accuracy: 0.7599
Epoch 8/10
2525/2525 [==============================] - 1308s 518ms/step - loss: 0.8262 - accuracy: 0.7745
Epoch 9/10
2525/2525 [==============================] - 1329s 526ms/step - loss: 0.7684 - accuracy: 0.7881
Epoch 10/10
2525/2525 [=========================

In [14]:
model.save('/model_saved/')

2022-03-11 13:08:20.089520: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [15]:
model.save_weights("/kaggle/working/saved_model.h5")